In [1]:
#install library
!pip install datasets transformers spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [2]:
import spacy
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading the dataset using HuggingFace's datasets library
dataset = load_dataset('cnn_dailymail', '3.0.0')

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
# Preprocessing function to clean the text
def preprocess(text):
    # Removing newline characters and extra spaces
    return text.replace('\n', ' ').strip()

In [5]:
# Applying preprocessing to the dataset
dataset = dataset.map(lambda x: {'article': preprocess(x['article'])})

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
# Loading spaCy's small English model
nlp = spacy.load("en_core_web_sm")

In [7]:
# Function for extractive summarization
def extractive_summarization(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    # Simple heuristic: Select top 3 longest sentences as summary
    summary = ' '.join(sorted(sentences, key=len, reverse=True)[:3])
    return summary

In [8]:
# Loading the pre-trained BART model and tokenizer from HuggingFace
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [9]:
# Creating a summarization pipeline
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

Device set to use cpu


In [10]:
# Function for abstractive summarization
def abstractive_summarization(text):
    # Generating summary with specified length constraints
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

In [11]:
# Selecting a sample article from the test set
sample_text = dataset['test'][0]['article']
sample_text

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [12]:
# Displaying the extractive summary
print("\nExtractive Summary:")
print(extractive_summarization(sample_text))


Extractive Summary:
"Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories.


In [13]:
# Displaying the abstractive summary
print("\nAbstractive Summary:")
print(abstractive_summarization(sample_text))


Abstractive Summary:
The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.


In [14]:
real_world_article = "SpaceX successfully launched another batch of Starlink satellites into orbit today. The Falcon 9 rocket lifted off from Cape Canaveral at 10:30 AM, marking the company's 30th launch of the year. This mission is part of SpaceX's ongoing effort to provide global high-speed internet coverage. The rocket's first stage landed on a drone ship in the Atlantic Ocean, making it the 15th successful recovery this year. Elon Musk stated that the company is on track to increase the frequency of launches in the coming months, with plans to deploy thousands more satellites."

In [15]:
# Testing with a real-world article
print("Real-world Article Extractive Summary:")
print(extractive_summarization(real_world_article))

Real-world Article Extractive Summary:
Elon Musk stated that the company is on track to increase the frequency of launches in the coming months, with plans to deploy thousands more satellites. The rocket's first stage landed on a drone ship in the Atlantic Ocean, making it the 15th successful recovery this year. The Falcon 9 rocket lifted off from Cape Canaveral at 10:30 AM, marking the company's 30th launch of the year.


In [16]:
print("Real-world Article Abstractive Summary:")
print(abstractive_summarization(real_world_article))

Your max_length is set to 130, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)


Real-world Article Abstractive Summary:
The Falcon 9 rocket lifted off from Cape Canaveral at 10:30 AM. This mission is part of SpaceX's ongoing effort to provide global high-speed internet coverage. The rocket's first stage landed on a drone ship in the Atlantic Ocean.
